# Using Parsed Data for Phrase Modelling

Executing bigram and trigram phrase modelling from the parsed sentences using konlpy

In [1]:
import sys

print(sys.executable)

/Library/Frameworks/Python.framework/Versions/3.6/bin/python3.6


In [2]:
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence

In [3]:
sentences_normalized_filepath = "bamboo_parsed.txt"

bigram_model_path = 'bigram_model'
trigram_model_path = 'trigram_model'

bigram_filepath = 'bamboo_bigram.txt'
trigram_filepath = 'bamboo_trigram.txt'

unigram_sentences = LineSentence(sentences_normalized_filepath)

bigram_model = Phrases(unigram_sentences)
bigram_model.save(bigram_model_path)

with open(bigram_filepath, 'w') as f:
        for unigram_sentence in unigram_sentences:
            bigram_sentence = bigram_model[unigram_sentence]
            f.write(' '.join(bigram_sentence) + '\n')
            
bigram_sentences = LineSentence(bigram_filepath)
trigram_model = Phrases(bigram_sentences)
trigram_model.save(trigram_model_path)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


Call the necessary parsed file to be modified 

In [33]:
from konlpy.tag import Okt
import csv

twitter = Okt()

results = []

data_path = 'posts_data_50.csv'

In [36]:
def clean_posts(post):
    lines = post[0].split("\n")
    for line in lines:
        temp_list = twitter.pos(line, norm=True, stem=True)
        r = []
        for word in temp_list:
            if not word[1] in ["Josa", "Eomi", "Punctuation"]:
                r.append(word[0])
        rl = (" ".join(r)).strip()
    return rl

In [37]:
with open(data_path) as f:
    data = csv.reader(f, delimiter=',')
    for row in data:
        results.append(clean_posts(row))

Prepping Data for LDA analysis

In [38]:
lda_filepath = 'lda_data.txt'

with open(lda_filepath, 'w') as f:
        for review_parsed in results:
            unigram_review = review_parsed
            bigram_review = bigram_model[unigram_review]
            trigram_review = trigram_model[bigram_review]

            f.write(' '.join(trigram_review) + '\n')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


# LDA Topic Modelling

performing LDA analysis

In [8]:
from gensim.corpora import Dictionary, MmCorpus
from gensim.models import word2vec

create dictionary for analysis

In [41]:
dict_made = False

dict_path = 'dictionary.dict'

if dict_made:
    dictionary = Dictionary.load(dict_path)
else:
    reviews_for_lda = word2vec.LineSentence(lda_filepath)
    dictionary = Dictionary(reviews_for_lda)
    dictionary.filter_extremes(no_below=10, no_above=0.4)
    dictionary.compactify()

    dictionary.save(dict_path)

In [ ]:
print(dictionary)

create corpus for analysis

In [43]:
corpus_path = 'corpus.mm'

def make_bow_corpus(filepath):
        """
        generator function to read in reviews from the file
        and output a bag-of-words represention of the text
        """
        for review in LineSentence(filepath):
            yield dictionary.doc2bow(review)
            
MmCorpus.serialize(corpus_path, make_bow_corpus(lda_filepath))

review_corpus = MmCorpus(corpus_path)

In [11]:
from gensim.models import LdaMulticore

In [ ]:
lda = LdaMulticore(review_corpus, num_topics=20, id2word=dictionary,  workers=2)
lda.save(lda_path)

In [ ]:
lda.show_topic(0)